In [32]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np

data = pd.read_csv('./data/train.csv', encoding='utf8')

data['arrival_is_estimated'] = data['arrival_is_estimated'].map({True: 1, False: 0}).fillna(0).astype(int)
def preprocess_duration_problem(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    # Convert arrival_time to datetime
    data['arrival_time'] = pd.to_datetime(data['arrival_time'], format='%H:%M:%S')
    
    # Calculate trip duration
    trip_durations = data.groupby('trip_id_unique').agg({
        'arrival_time': ['min', 'max']
    })
    trip_durations.columns = ['start_time', 'end_time']
    trip_durations['trip_duration_minutes'] = (trip_durations['end_time'] - trip_durations['start_time']).dt.total_seconds() / 60
    
    # Feature engineering
    features = pd.DataFrame()
    features['station_cnt'] = data.groupby("trip_id_unique")["trip_id_unique_station"].nunique()
    features['total_passenger'] = data.groupby("trip_id_unique")["passengers_up"].sum()
    features['mean_passenger'] = data.groupby("trip_id_unique")["passengers_up"].mean()
    features['mean_passenger_continue'] = data.groupby("trip_id_unique")["passengers_continue"].mean()
    features['start_hour'] = data.groupby("trip_id_unique")["arrival_time"].min().dt.hour
    
    # Add cluster, direction, and mekadem_nipuach_luz
    features = features.merge(data[["trip_id_unique", "cluster", "direction", "mekadem_nipuach_luz"]].drop_duplicates(), on="trip_id_unique")
    
    # Encode cluster
    label_encoder = LabelEncoder()
    features['cluster'] = label_encoder.fit_transform(features['cluster'])
    
    # Add station name features
    station_concat = data.groupby("trip_id_unique")["station_name"].agg(lambda x: ', '.join(x)).reset_index()
    for i, word in enumerate(WORDS_WEIGHT, start=1):
        features[f'station_{word}'] = station_concat['station_name'].str.contains(word, regex=False).astype(int)
    
    # Calculate approximate line length
    features['line_length_approx'] = data.groupby('trip_id_unique').apply(calculate_approx_line_length)
    
    # Merge trip durations
    features = features.merge(trip_durations['trip_duration_minutes'], on='trip_id_unique')
    
    # Separate features and target
    X = features.drop(['trip_duration_minutes', 'trip_id_unique'], axis=1)
    y = features['trip_duration_minutes']
    
    return X, y

Mean Squared Error: 18350.607679928158
R-squared Score: -0.020831563653008534
         feature  importance
1    end_station    0.810949
0  start_station    0.189051


In [19]:
data = pd.read_csv('./data/REAL_DATA.csv',encoding='ISO-8859-8')

trip_duration_train = data.groupby('trip_id_unique_station')['arrival_time'].max() - data.groupby('trip_id_unique_station')['arrival_time'].min()



TypeError: unsupported operand type(s) for -: 'str' and 'str'